In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
#como el navegador a usar es chrome, hemos descargado ya el driver de la página, guardamos la ruta en una variable
#chrome_path = 'chromedriver_linux64/chromedriver'
#driver = webdriver.Chrome(executable_path = chrome_path)

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}

¿Qué queremos obtener de la página?
1. Nombre del producto
2. Precio (?)
3. Detalles
4. Ingredientes

## 1 OBTENER TODAS LAS URLS DE LOS PRODUCTOS

Para ello, utilizaremos bucles que iteren sobre los elementos de la página.
* Variable global cleansers_url, es el contenedor de todas las urls.
* Los links están dentro de un tag 'a', como sólo nos interesan aquellas que tengan href, pues href=True. 
* Se ha agregado class='product-image" porque si no, el bucle cogía todas las urls (wishlist, url de la imagen etc), y solo nos interesan los productos.
* Append cada uno de los links al contenedor.

In [ ]:
toner_url = []

In [4]:
for i in range (1,68):
    url = f'https://www.stylevana.com/en_EU/skincare/face-care/toner-mist.html?p={i}' 
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    toner_list = soup.find_all('div', class_='product-item-info')
    for item in toner_list:
        for link in item.find_all('a', class_='product-image', href=True): 
            toner_url.append(link['href'])
print(len(toner_url))

792


with open("cleansers_url.txt", "w") as output:
    output.write(str(cleansers_url))

## 2 OBTENER LA INFO DE LA PÁGINA PROPIA DEL PRODUCTO
Info que necesitamos:
* Nombre del producto
* Details
* Ingredients

* Details e Ingredients se encuentran dentro de un grupo <div class='panel-group'

In [5]:
toners = []

In [6]:
for link in toner_url[1:793]:
    r = requests.get(link, headers=headers) #para cada link en el listado, hacemos una requests
    soup = BeautifulSoup(r.content, 'lxml')
    #usamos try, except porque hay ocasiones en las que no hay info sobre los ingredientes
    try:
        name = soup.find('h1', class_='product-name-h1').text
        details = (soup.find('div', id='product-view-details').text.strip()) # details
        ingredients = soup.find_all('div', id="product-view-ingredients") #ingredients
        for item in ingredients:
            ingredients = item.find('div', class_='panel-body').text.strip()
    except:
        name = 'no info'
        details = 'no info'
        ingredients = 'no info'
        
    toners_info = {
    'name': name,
    'details': details,
    'ingredients': ingredients}
    
    toners.append(toners_info)

In [7]:
print(len(toners))

791


## 3. PASARLO A DATAFRAME

In [8]:
toners_df = pd.DataFrame(toners)
toners_df

,name,details,ingredients
0,SOME BY MI - AHA-BHA-PHA 30 Days Miracle Toner...,"Usage: Day, Night\n\n\nProduct Type: Toner & M...",[]
1,COSRX - Centella Water Alcohol-Free Toner 150ml,"Cruelty Free: Yes\n\n\nUsage: Day, Night\n\n\n...","Water, Centella Asiatica Leaf Water, Butylene ..."
2,Haruharu WONDER - Black Rice Hyaluronic Toner ...,"Usage: Day, Night\n\n\nProduct Type: Toner & M...","After washing the face, wet a cotton pad with ..."
3,Pyunkang Yul - Essence Toner,"Cruelty Free: Yes\n\n\nUsage: Day, Night\n\n\n...",[]
4,Sioris - Time Is Running Out Mist,"Cruelty Free: Yes\n\n\nUsage: Day, Night\n\n\n...",[]
...,...,...,...
786,NATURE REPUBLIC - Pore Away Clear Toner - 300ml,"Usage: Day, Night\n\n\nProduct Type: Toner & M...",[]
787,The Saem - Eco Energy Fresh Toner - 150ml,"Usage: Day, Night\n\n\nProduct Type: Toner & M...",[]
788,NEOGEN Dermalogy - Pore Refine Toner - 150ml,"Cruelty Free: Yes\n\n\nUsage: Day, Night\n\n\n...",[]
789,VANT 36.5 - AHA 5% Toner - 100ml,"Usage: Day, Night\n\n\nProduct Type: Toner & M...",[]


In [9]:
toners = toners_df.to_csv('data/toners.csv')